In [ ]:
import torch
from data.names_dataset import NamesDataset
from torch.utils.data import DataLoader

names_dataset = NamesDataset(data_folder="../datasets/names")


train_dataset, test_dataset = torch.utils.data.random_split(names_dataset, [0.85, 0.15])
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

In [ ]:
BATCH_SIZE = 64


def collate_fn(batch) -> tuple[list[torch.Tensor], list[torch.Tensor]]:
    inputs, labels = zip(*batch)
    return list(inputs), list(labels)


train_dataloader = DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn
)

test_dataloader = DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)

train_features, train_labels = next(iter(train_dataloader))
print(train_features)
print(train_labels)
print(len(train_features))
print(len(train_labels))
print(train_features[0].size())
print(train_labels[0].size())


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class NamesClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NamesClassifier, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_size, hidden_size=hidden_size, batch_first=True
        )
        self.h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x is of shape (batch_size, seq_length, input_size)
        output, hidden = self.rnn(x)
        output = self.h2o(hidden[0])
        # output is of shape (batch_size, output_size)
        output = F.log_softmax(output, dim=1)
        return output


rnn = NamesClassifier(
    input_size=len(names_dataset.index_to_token),
    hidden_size=128,
    output_size=len(names_dataset.countries),
)
print(rnn)

tensor = names_dataset.name_to_tensor("O'Connor")
print(f"{tensor.size()} {tensor.dtype}")
output = rnn(tensor)
print(f"{output=}")
print(names_dataset.tensor_to_country(output))

In [ ]:
def train(
    model: nn.Module,
    dataloader: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
):
    model.train()
    total_loss = 0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.argmax(dim=1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


def evaluate(model: nn.Module, dataloader: DataLoader, criterion: nn.Module):
    model.eval()
    total_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.argmax(dim=1))
            total_loss += loss.item()
            predictions = outputs.argmax(dim=1)
            correct += (predictions == labels.argmax(dim=1)).sum().item()